In [4]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import pandas as pd
from typing import Dict, Tuple, Optional, List

# -------------------------
# 1) PCA形态度量
# -------------------------
def pca_spectrum_metrics(pts: np.ndarray) -> Dict[str, float]:
    """
    输入: (N,3) 点云
    返回: PCA谱相关度量:
      - eigvals: 降序 [lambda1, lambda2, lambda3]
      - r = lambda2/lambda1 (主方向分散度)
      - sphericity = lambda3/lambda1 (薄板性, 越小越薄)
      - linearity  = (lambda1 - lambda2)/lambda1
      - planarity  = (lambda2 - lambda3)/lambda1
    """
    if pts.shape[0] < 3:
        return dict(lambda1=np.nan, lambda2=np.nan, lambda3=np.nan,
                    ratio_r=np.nan, sphericity=np.nan, linearity=np.nan, planarity=np.nan)
    # 中心化
    X = pts - pts.mean(axis=0, keepdims=True)
    C = np.cov(X.T)  # 3x3
    eigvals, _ = np.linalg.eigh(C)  # 升序
    eigvals = np.sort(eigvals)[::-1]  # 降序
    l1, l2, l3 = eigvals
    # 数值保护
    l1 = float(max(l1, 1e-12))
    l2 = float(max(l2, 0.0))
    l3 = float(max(l3, 0.0))
    ratio_r = l2 / l1
    sphericity = l3 / l1
    linearity  = (l1 - l2) / l1
    planarity  = (l2 - l3) / l1
    return dict(lambda1=l1, lambda2=l2, lambda3=l3,
                ratio_r=ratio_r, sphericity=sphericity,
                linearity=linearity, planarity=planarity)

# -------------------------
# 2) 2D 投影 + 连通性证据（DBSCAN 多阈值探测）
# -------------------------
def two_d_connectivity_evidence(pts: np.ndarray,
                                min_samples_frac: float = 0.01,
                                knn_k: int = 6,
                                eps_scale_grid: Tuple[float, ...] = (0.8, 1.0, 1.2),
                                min_child_frac: float = 0.10) -> Dict[str, object]:
    """
    将3D点投影到前两主成分的2D平面，在2D上用DBSCAN多阈值探测“是否存在>=2个稳定子团”。

    返回:
      - stable_multi: bool 是否有稳定多团证据
      - n_clusters_grid: list 每个 eps_scale 下的簇数 (>=2 视为可分)
      - best_labels: 一个代表性的DBSCAN标签 (用于查看)
    """
    N = pts.shape[0]
    if N < 10:
        return dict(stable_multi=False, n_clusters_grid=[], best_labels=None)

    # 2D PCA 投影
    pca = PCA(n_components=2)
    P2 = pca.fit_transform(pts)

    # min_samples（随规模变化）
    min_samples = max(2, int(np.ceil(N * min_samples_frac)))

    # 用 kNN 第 k 邻距的中位数做eps基准
    nn = NearestNeighbors(n_neighbors=min(knn_k, N)).fit(P2)
    dists, _ = nn.kneighbors(P2)
    base_eps = float(np.median(dists[:, -1]))

    n_clusters_grid = []
    best_labels = None
    multi_count = 0

    for s in eps_scale_grid:
        eps = max(1e-6, s * base_eps)
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(P2)
        labels = db.labels_
        # 有效簇（忽略噪声 -1）
        uniq = [c for c in np.unique(labels) if c >= 0]
        # 每个簇的最小规模约束，防止把一个簇分成一堆小碎片也算“可分”
        good_clusters = [c for c in uniq if np.sum(labels == c) >= max(2, int(N * min_child_frac))]
        n_clusters_grid.append(len(good_clusters))
        if len(good_clusters) >= 2:
            multi_count += 1
            best_labels = labels  # 记录一个代表性的标签

    # 稳定性：多次 eps 缩放中，至少有一半比例认为可分 ⇒ 认为“稳定可分”
    stable_multi = (multi_count >= (len(eps_scale_grid) // 2 + 1))

    return dict(stable_multi=stable_multi,
                n_clusters_grid=n_clusters_grid,
                best_labels=best_labels)

# -------------------------
# 3) 组合判据：PCA谱 + 2D连通性
# -------------------------
def need_refine_cluster(pts: np.ndarray,
                        s_th: float = 0.08,
                        r_th: float = 0.75,
                        min_samples_frac: float = 0.01,
                        knn_k: int = 6,
                        eps_scale_grid: Tuple[float, ...] = (0.8, 1.0, 1.2),
                        min_child_frac: float = 0.10) -> Dict[str, object]:
    """
    判定某个簇是否需要“进一步细分”：
      1) 薄板性:   sphericity < s_th
      2) 主方向分散: ratio_r > r_th
      3) 2D 投影稳定多团: two_d_connectivity_evidence.stable_multi == True
    三者同时成立 ⇒ 需要细分
    """
    metrics = pca_spectrum_metrics(pts)
    sphericity = metrics['sphericity']
    ratio_r    = metrics['ratio_r']

    two_d = two_d_connectivity_evidence(
        pts,
        min_samples_frac=min_samples_frac,
        knn_k=knn_k,
        eps_scale_grid=eps_scale_grid,
        min_child_frac=min_child_frac
    )

    cond_thin   = (np.isfinite(sphericity) and sphericity < s_th)
    cond_spread = (np.isfinite(ratio_r)    and ratio_r    > r_th)
    cond_2d     = two_d['stable_multi'] is True

    return dict(
        need_refine=bool(cond_thin and cond_spread and cond_2d),
        sphericity=sphericity,
        ratio_r=ratio_r,
        n_clusters_grid=two_d['n_clusters_grid'],
        stable_multi=two_d['stable_multi']
    )

# -------------------------
# 4) 主函数：读取 .txt 并逐簇评估
# -------------------------
def analyze_file_for_refine(txt_path: str,
                            ignore_label: Optional[int] = None,
                            min_points_per_cluster: int = 30,
                            s_th: float = 0.08,
                            r_th: float = 0.75,
                            min_samples_frac: float = 0.01,
                            knn_k: int = 6,
                            eps_scale_grid: Tuple[float, ...] = (0.8, 1.0, 1.2),
                            min_child_frac: float = 0.10) -> pd.DataFrame:
    """
    输入：.txt（至少四列：x y z label）
    输出：DataFrame，每一行一个簇，字段包含：
      label, n_points, sphericity, ratio_r, n_clusters_grid, stable_multi, need_refine
    """
    arr = np.loadtxt(txt_path)
    if arr.shape[1] < 4:
        raise ValueError("文件至少需要 4 列：x y z label")

    xyz = arr[:, :3]
    labels = arr[:, 3].astype(int)

    results = []
    for lab in np.unique(labels):
        if ignore_label is not None and lab == ignore_label:
            continue
        mask = labels == lab
        pts = xyz[mask]
        if pts.shape[0] < min_points_per_cluster:
            # 太小的簇：不做判定，默认不细分
            results.append(dict(label=int(lab), n_points=int(pts.shape[0]),
                                sphericity=np.nan, ratio_r=np.nan,
                                n_clusters_grid=[], stable_multi=False,
                                need_refine=False, reason="too_small"))
            continue

        r = need_refine_cluster(
            pts,
            s_th=s_th, r_th=r_th,
            min_samples_frac=min_samples_frac,
            knn_k=knn_k,
            eps_scale_grid=eps_scale_grid,
            min_child_frac=min_child_frac
        )
        results.append(dict(
            label=int(lab),
            n_points=int(pts.shape[0]),
            sphericity=float(r['sphericity']),
            ratio_r=float(r['ratio_r']),
            n_clusters_grid=r['n_clusters_grid'],
            stable_multi=bool(r['stable_multi']),
            need_refine=bool(r['need_refine']),
            reason="pca+2d"
        ))

    df = pd.DataFrame(results).sort_values("label").reset_index(drop=True)
    return df

# -------------------------
# 5) 使用示例（在Notebook里修改路径与参数）
# -------------------------

# 配置
txt_path = "/home/zero/mnt/sda6/split_data_instance/ptomato/quk++4/032211-11_qspp.txt"  # 修改为你的 .txt 文件路径
IGNORE_LABEL = None      # 若茎秆标签为1，可忽略；不忽略则设为 None
MIN_CLUSTER_SIZE = 60 # 小于该点数的簇不做判定
S_TH = 0.15           # 薄板阈值: sphericity < S_TH
R_TH = 0.6           # 主方向分散阈值: ratio_r > R_TH

# 运行
try:
    df_result = analyze_file_for_refine(
        txt_path,
        ignore_label=IGNORE_LABEL,
        min_points_per_cluster=MIN_CLUSTER_SIZE,
        s_th=S_TH,
        r_th=R_TH,
        min_samples_frac=0.01,     # DBSCAN 的 min_samples ~ 1%N
        knn_k=6,                   # eps 基准：第6近邻中位数
        eps_scale_grid=(0.8,1.0,1.2), # 多阈值探测
        min_child_frac=0.10        # 子簇至少占该簇10%点
    )
    # 打印结果
    for _, row in df_result.iterrows():
        flag = "需要进一步分割" if row["need_refine"] else "无需细分"
        print(f"簇 {int(row['label'])}: {flag} | n={int(row['n_points'])}, "
              f"sphericity={row['sphericity']:.4f} | ratio_r={row['ratio_r']:.4f} | "
              f"2D可分簇数@多阈值={row['n_clusters_grid']} (稳定可分={row['stable_multi']})")
    # 也可以查看 DataFrame
    df_result
except Exception as e:
    print("运行出错：", e)

簇 0: 无需细分 | n=58, sphericity=nan | ratio_r=nan | 2D可分簇数@多阈值=[] (稳定可分=False)
簇 1: 无需细分 | n=174, sphericity=0.3275 | ratio_r=0.7210 | 2D可分簇数@多阈值=[2, 1, 1] (稳定可分=False)
簇 2: 无需细分 | n=147, sphericity=0.0223 | ratio_r=0.1896 | 2D可分簇数@多阈值=[2, 1, 1] (稳定可分=False)
簇 3: 无需细分 | n=177, sphericity=0.0128 | ratio_r=0.2479 | 2D可分簇数@多阈值=[2, 1, 1] (稳定可分=False)
簇 4: 无需细分 | n=155, sphericity=0.0221 | ratio_r=0.3979 | 2D可分簇数@多阈值=[2, 2, 1] (稳定可分=True)
簇 5: 无需细分 | n=186, sphericity=0.0170 | ratio_r=0.2979 | 2D可分簇数@多阈值=[1, 1, 1] (稳定可分=False)
簇 6: 无需细分 | n=286, sphericity=0.0259 | ratio_r=0.3254 | 2D可分簇数@多阈值=[1, 1, 1] (稳定可分=False)
簇 7: 无需细分 | n=108, sphericity=0.0123 | ratio_r=0.0837 | 2D可分簇数@多阈值=[2, 2, 2] (稳定可分=True)
簇 8: 无需细分 | n=305, sphericity=0.0255 | ratio_r=0.3529 | 2D可分簇数@多阈值=[2, 1, 1] (稳定可分=False)
簇 9: 无需细分 | n=87, sphericity=0.0331 | ratio_r=0.4732 | 2D可分簇数@多阈值=[1, 1, 1] (稳定可分=False)
簇 10: 无需细分 | n=78, sphericity=0.0130 | ratio_r=0.2449 | 2D可分簇数@多阈值=[2, 1, 1] (稳定可分=False)
簇 11: 无需细分 | n=364, spheric

In [5]:
# === Jupyter 单元格：逐簇 PCA 主成分方向与谱特征 ===

import numpy as np
import pandas as pd

# --------- 配置区（按需修改）---------
txt_path = "/home/zero/mnt/sda6/split_data_instance/ptomato/quk++4/032211-11_qspp.txt"  # 你的 .txt 文件路径
IGNORE_LABEL = None    # 若想忽略某类（如茎秆=1），则设置为 1；不忽略则设 None
MIN_POINTS = 50        # 少于该点数的簇不做PCA（或只做占位）
DECIMALS = 4           # 打印小数位
SAVE_CSV = False       # 若需要保存结果为CSV，改为 True
CSV_PATH = "/home/zero/mnt/sda6/split_data_instance/ptomato/quk++4/cluster_pca_summary11.csv"
# ------------------------------------
def pca_cluster(points: np.ndarray):
    """
    对 (N,3) 点云做 PCA，返回（λ按降序）特征值/向量与形态指标。
    """
    if points.shape[0] < 3:
        return dict(lambda1=np.nan, lambda2=np.nan, lambda3=np.nan,
                    v1=np.array([np.nan, np.nan, np.nan]),
                    v2=np.array([np.nan, np.nan, np.nan]),
                    v3=np.array([np.nan, np.nan, np.nan]),
                    ratio_r=np.nan, sphericity=np.nan,
                    linearity=np.nan, planarity=np.nan)
    X = points - points.mean(axis=0, keepdims=True)
    C = np.cov(X.T)                       # 3x3 协方差
    eigvals, eigvecs = np.linalg.eigh(C)  # 特征分解，eigvals 升序
    order = np.argsort(eigvals)[::-1]     # 按降序排序
    eigvals = eigvals[order]
    eigvecs = eigvecs[:, order]
    l1, l2, l3 = [float(max(v, 0.0)) for v in eigvals]
    # 防除零
    l1_eps = l1 if l1 > 1e-12 else 1e-12
    ratio_r    = l2 / l1_eps
    sphericity = l3 / l1_eps
    linearity  = (l1 - l2) / l1_eps
    planarity  = (l2 - l3) / l1_eps
    # 主方向向量（列向量）
    v1 = eigvecs[:, 0]
    v2 = eigvecs[:, 1]
    v3 = eigvecs[:, 2]
    return dict(lambda1=l1, lambda2=l2, lambda3=l3,
                v1=v1, v2=v2, v3=v3,
                ratio_r=ratio_r, sphericity=sphericity,
                linearity=linearity, planarity=planarity)

# 读取数据
arr = np.loadtxt(txt_path)
if arr.shape[1] < 4:
    raise ValueError("文件至少应包含 4 列：x y z label")

xyz = arr[:, :3]
labels = arr[:, 3].astype(int)
# 逐簇统计
rows = []
for lab in np.unique(labels):
    if IGNORE_LABEL is not None and lab == IGNORE_LABEL:
        continue
    pts = xyz[labels == lab]
    n = pts.shape[0]

    if n < MIN_POINTS:
        rows.append(dict(
            label=int(lab), n_points=int(n),
            cx=float(np.mean(pts[:,0])) if n>0 else np.nan,
            cy=float(np.mean(pts[:,1])) if n>0 else np.nan,
            cz=float(np.mean(pts[:,2])) if n>0 else np.nan,
            lambda1=np.nan, lambda2=np.nan, lambda3=np.nan,
            ratio_r=np.nan, sphericity=np.nan,
            linearity=np.nan, planarity=np.nan,
            v1x=np.nan, v1y=np.nan, v1z=np.nan,
            v2x=np.nan, v2y=np.nan, v2z=np.nan,
            v3x=np.nan, v3y=np.nan, v3z=np.nan,
            note="too_small"
        ))
        continue
    res = pca_cluster(pts)
    v1, v2, v3 = res['v1'], res['v2'], res['v3']
    rows.append(dict(
        label=int(lab), n_points=int(n),
        cx=float(np.mean(pts[:,0])), cy=float(np.mean(pts[:,1])), cz=float(np.mean(pts[:,2])),
        lambda1=res['lambda1'], lambda2=res['lambda2'], lambda3=res['lambda3'],
        ratio_r=res['ratio_r'], sphericity=res['sphericity'],
        linearity=res['linearity'], planarity=res['planarity'],
        v1x=v1[0], v1y=v1[1], v1z=v1[2],
        v2x=v2[0], v2y=v2[1], v2z=v2[2],
        v3x=v3[0], v3y=v3[1], v3z=v3[2],
        note=""
    ))
df = pd.DataFrame(rows).sort_values("label").reset_index(drop=True)

# 逐簇打印（主方向与指标）
np.set_printoptions(precision=DECIMALS, suppress=True)
for _, r in df.iterrows():
    lab = int(r['label']); n = int(r['n_points'])
    if r['note'] == "too_small":
        print(f"簇 {lab}: n={n} (点数过少，跳过PCA)")
        continue
    v1 = np.array([r['v1x'], r['v1y'], r['v1z']])
    v2 = np.array([r['v2x'], r['v2y'], r['v2z']])
    v3 = np.array([r['v3x'], r['v3y'], r['v3z']])
    print(f"簇 {lab}: n={n}")
    print(f"  主方向 v1 = {np.round(v1, DECIMALS)}")
    print(f"  次方向 v2 = {np.round(v2, DECIMALS)}")
    print(f"  法向  v3 = {np.round(v3, DECIMALS)}")
    print(f"  λ = [{r['lambda1']:.{DECIMALS}f}, {r['lambda2']:.{DECIMALS}f}, {r['lambda3']:.{DECIMALS}f}] "
          f"| ratio_r(λ2/λ1)={r['ratio_r']:.{DECIMALS}f}, sphericity(λ3/λ1)={r['sphericity']:.{DECIMALS}f}, "
          f"linearity={r['linearity']:.{DECIMALS}f}, planarity={r['planarity']:.{DECIMALS}f}")

# 显示汇总表（Jupyter 会渲染）
df_rounded = df.copy()
for col in df_rounded.columns:
    if df_rounded[col].dtype.kind == 'f':
        df_rounded[col] = df_rounded[col].round(DECIMALS)
df_rounded

# 如需保存CSV，取消下一行注释
# if SAVE_CSV:
#     df_rounded.to_csv(CSV_PATH, index=False, encoding="utf-8-sig")
#     print("已保存：", CSV_PATH)

簇 0: n=58
  主方向 v1 = [-0.5029  0.6435  0.5771]
  次方向 v2 = [ 0.6082  0.7379 -0.2927]
  法向  v3 = [-0.6142  0.2038 -0.7624]
  λ = [0.0015, 0.0003, 0.0001] | ratio_r(λ2/λ1)=0.1671, sphericity(λ3/λ1)=0.0614, linearity=0.8329, planarity=0.1057
簇 1: n=174
  主方向 v1 = [-0.5117  0.8375  0.1916]
  次方向 v2 = [ 0.8454  0.5306 -0.0614]
  法向  v3 = [ 0.1531 -0.1306  0.9795]
  λ = [0.0028, 0.0020, 0.0009] | ratio_r(λ2/λ1)=0.7210, sphericity(λ3/λ1)=0.3275, linearity=0.2790, planarity=0.3935
簇 2: n=147
  主方向 v1 = [ 0.054  -0.8113 -0.5821]
  次方向 v2 = [ 0.9842  0.1418 -0.1063]
  法向  v3 = [ 0.1688 -0.5672  0.8061]
  λ = [0.0044, 0.0008, 0.0001] | ratio_r(λ2/λ1)=0.1896, sphericity(λ3/λ1)=0.0223, linearity=0.8104, planarity=0.1673
簇 3: n=177
  主方向 v1 = [ 0.2003 -0.5812 -0.7887]
  次方向 v2 = [ 0.8862  0.4507 -0.107 ]
  法向  v3 = [ 0.4177 -0.6775  0.6054]
  λ = [0.0042, 0.0010, 0.0001] | ratio_r(λ2/λ1)=0.2479, sphericity(λ3/λ1)=0.0128, linearity=0.7521, planarity=0.2351
簇 4: n=155
  主方向 v1 = [-0.6334  0.7443  0.211

,label,n_points,cx,cy,cz,lambda1,lambda2,lambda3,ratio_r,sphericity,...,v1x,v1y,v1z,v2x,v2y,v2z,v3x,v3y,v3z,note
0,0,58,0.0883,0.0259,0.2707,0.0015,0.0003,0.0001,0.1671,0.0614,...,-0.5029,0.6435,0.5771,0.6082,0.7379,-0.2927,-0.6142,0.2038,-0.7624,
1,1,174,0.0064,-0.0545,0.2443,0.0028,0.0020,0.0009,0.7210,0.3275,...,-0.5117,0.8375,0.1916,0.8454,0.5306,-0.0614,0.1531,-0.1306,0.9795,
2,2,147,0.2189,-0.0456,0.2454,0.0044,0.0008,0.0001,0.1896,0.0223,...,0.0540,-0.8113,-0.5821,0.9842,0.1418,-0.1063,0.1688,-0.5672,0.8061,
3,3,177,0.3390,-0.0009,0.2188,0.0042,0.0010,0.0001,0.2479,0.0128,...,0.2003,-0.5812,-0.7887,0.8862,0.4507,-0.1070,0.4177,-0.6775,0.6054,
4,4,155,0.1631,0.1251,0.3308,0.0032,0.0013,0.0001,0.3979,0.0221,...,-0.6334,0.7443,0.2117,0.7655,0.6426,0.0310,0.1130,-0.1817,0.9768,
5,5,186,0.3212,0.1978,0.2407,0.0043,0.0013,0.0001,0.2979,0.0170,...,0.0275,-0.7901,0.6123,0.9102,-0.2334,-0.3420,0.4132,0.5667,0.7128,
6,6,286,0.4258,0.1082,0.1006,0.0062,0.0020,0.0002,0.3254,0.0259,...,-0.2307,0.0372,0.9723,0.1003,0.9949,-0.0143,0.9679,-0.0942,0.2332,
7,7,108,0.0008,-0.1757,0.1626,0.0060,0.0005,0.0001,0.0837,0.0123,...,-0.9929,0.0245,0.1167,-0.0558,-0.9603,-0.2732,0.1054,-0.2778,0.9548,
8,8,305,-0.1596,-0.1021,0.2732,0.0068,0.0024,0.0002,0.3529,0.0255,...,-0.8216,-0.4278,-0.3768,-0.5206,0.8323,0.1902,-0.2323,-0.3524,0.9066,
9,9,87,-0.0777,0.1759,0.2744,0.0016,0.0008,0.0001,0.4732,0.0331,...,0.5473,0.7624,-0.3454,-0.5641,0.6409,0.5206,-0.6182,0.0901,-0.7808,


In [6]:
# === Jupyter 单元格：逐簇 PCA 主成分方向与谱特征 ===

import numpy as np
import pandas as pd

# --------- 配置区（按需修改）---------
txt_path_19 = "/home/zero/mnt/sda6/split_data_instance/ptomato/quk++4/032211-19_qspp.txt"  # 你的 .txt 文件路径
IGNORE_LABEL = None    # 若想忽略某类（如茎秆=1），则设置为 1；不忽略则设 None
MIN_POINTS = 50        # 少于该点数的簇不做PCA（或只做占位）
DECIMALS = 4           # 打印小数位
SAVE_CSV = False       # 若需要保存结果为CSV，改为 True
CSV_PATH_19 = "/home/zero/mnt/sda6/split_data_instance/ptomato/quk++4/cluster_pca_summary19.csv"
# ------------------------------------
def pca_cluster(points: np.ndarray):
    """
    对 (N,3) 点云做 PCA，返回（λ按降序）特征值/向量与形态指标。
    """
    if points.shape[0] < 3:
        return dict(lambda1=np.nan, lambda2=np.nan, lambda3=np.nan,
                    v1=np.array([np.nan, np.nan, np.nan]),
                    v2=np.array([np.nan, np.nan, np.nan]),
                    v3=np.array([np.nan, np.nan, np.nan]),
                    ratio_r=np.nan, sphericity=np.nan,
                    linearity=np.nan, planarity=np.nan)
    X = points - points.mean(axis=0, keepdims=True)
    C = np.cov(X.T)                       # 3x3 协方差
    eigvals, eigvecs = np.linalg.eigh(C)  # 特征分解，eigvals 升序
    order = np.argsort(eigvals)[::-1]     # 按降序排序
    eigvals = eigvals[order]
    eigvecs = eigvecs[:, order]
    l1, l2, l3 = [float(max(v, 0.0)) for v in eigvals]
    # 防除零
    l1_eps = l1 if l1 > 1e-12 else 1e-12
    ratio_r    = l2 / l1_eps
    sphericity = l3 / l1_eps
    linearity  = (l1 - l2) / l1_eps
    planarity  = (l2 - l3) / l1_eps
    # 主方向向量（列向量）
    v1 = eigvecs[:, 0]
    v2 = eigvecs[:, 1]
    v3 = eigvecs[:, 2]
    return dict(lambda1=l1, lambda2=l2, lambda3=l3,
                v1=v1, v2=v2, v3=v3,
                ratio_r=ratio_r, sphericity=sphericity,
                linearity=linearity, planarity=planarity)

# 读取数据
arr = np.loadtxt(txt_path_19)
if arr.shape[1] < 4:
    raise ValueError("文件至少应包含 4 列：x y z label")

xyz = arr[:, :3]
labels = arr[:, 3].astype(int)
# 逐簇统计
rows = []
for lab in np.unique(labels):
    if IGNORE_LABEL is not None and lab == IGNORE_LABEL:
        continue
    pts = xyz[labels == lab]
    n = pts.shape[0]

    if n < MIN_POINTS:
        rows.append(dict(
            label=int(lab), n_points=int(n),
            cx=float(np.mean(pts[:,0])) if n>0 else np.nan,
            cy=float(np.mean(pts[:,1])) if n>0 else np.nan,
            cz=float(np.mean(pts[:,2])) if n>0 else np.nan,
            lambda1=np.nan, lambda2=np.nan, lambda3=np.nan,
            ratio_r=np.nan, sphericity=np.nan,
            linearity=np.nan, planarity=np.nan,
            v1x=np.nan, v1y=np.nan, v1z=np.nan,
            v2x=np.nan, v2y=np.nan, v2z=np.nan,
            v3x=np.nan, v3y=np.nan, v3z=np.nan,
            note="too_small"
        ))
        continue
    res = pca_cluster(pts)
    v1, v2, v3 = res['v1'], res['v2'], res['v3']
    rows.append(dict(
        label=int(lab), n_points=int(n),
        cx=float(np.mean(pts[:,0])), cy=float(np.mean(pts[:,1])), cz=float(np.mean(pts[:,2])),
        lambda1=res['lambda1'], lambda2=res['lambda2'], lambda3=res['lambda3'],
        ratio_r=res['ratio_r'], sphericity=res['sphericity'],
        linearity=res['linearity'], planarity=res['planarity'],
        v1x=v1[0], v1y=v1[1], v1z=v1[2],
        v2x=v2[0], v2y=v2[1], v2z=v2[2],
        v3x=v3[0], v3y=v3[1], v3z=v3[2],
        note=""
    ))
df = pd.DataFrame(rows).sort_values("label").reset_index(drop=True)

# 逐簇打印（主方向与指标）
np.set_printoptions(precision=DECIMALS, suppress=True)
for _, r in df.iterrows():
    lab = int(r['label']); n = int(r['n_points'])
    if r['note'] == "too_small":
        print(f"簇 {lab}: n={n} (点数过少，跳过PCA)")
        continue
    v1 = np.array([r['v1x'], r['v1y'], r['v1z']])
    v2 = np.array([r['v2x'], r['v2y'], r['v2z']])
    v3 = np.array([r['v3x'], r['v3y'], r['v3z']])
    print(f"簇 {lab}: n={n}")
    print(f"  主方向 v1 = {np.round(v1, DECIMALS)}")
    print(f"  次方向 v2 = {np.round(v2, DECIMALS)}")
    print(f"  法向  v3 = {np.round(v3, DECIMALS)}")
    print(f"  λ = [{r['lambda1']:.{DECIMALS}f}, {r['lambda2']:.{DECIMALS}f}, {r['lambda3']:.{DECIMALS}f}] "
          f"| ratio_r(λ2/λ1)={r['ratio_r']:.{DECIMALS}f}, sphericity(λ3/λ1)={r['sphericity']:.{DECIMALS}f}, "
          f"linearity={r['linearity']:.{DECIMALS}f}, planarity={r['planarity']:.{DECIMALS}f}")

# 显示汇总表（Jupyter 会渲染）
df_rounded = df.copy()
for col in df_rounded.columns:
    if df_rounded[col].dtype.kind == 'f':
        df_rounded[col] = df_rounded[col].round(DECIMALS)
df_rounded

# 如需保存CSV，取消下一行注释
# if SAVE_CSV:
#     df_rounded.to_csv(CSV_PATH, index=False, encoding="utf-8-sig")
#     print("已保存：", CSV_PATH)

簇 0: n=1090
  主方向 v1 = [-0.719  -0.6667 -0.1961]
  次方向 v2 = [ 0.6638 -0.5753 -0.4779]
  法向  v3 = [ 0.2058 -0.4738  0.8562]
  λ = [0.0193, 0.0147, 0.0001] | ratio_r(λ2/λ1)=0.7633, sphericity(λ3/λ1)=0.0069, linearity=0.2367, planarity=0.7565
簇 1: n=326
  主方向 v1 = [-0.8675 -0.0494  0.495 ]
  次方向 v2 = [0.0238 0.9898 0.1403]
  法向  v3 = [-0.4969  0.1335 -0.8575]
  λ = [0.0055, 0.0022, 0.0001] | ratio_r(λ2/λ1)=0.3956, sphericity(λ3/λ1)=0.0118, linearity=0.6044, planarity=0.3838
簇 2: n=220
  主方向 v1 = [-0.9856 -0.1378 -0.0983]
  次方向 v2 = [-0.1624  0.9335  0.3197]
  法向  v3 = [-0.0477 -0.331   0.9424]
  λ = [0.0042, 0.0018, 0.0000] | ratio_r(λ2/λ1)=0.4337, sphericity(λ3/λ1)=0.0054, linearity=0.5663, planarity=0.4282
簇 3: n=562
  主方向 v1 = [-0.5435 -0.7706 -0.3329]
  次方向 v2 = [-0.8334  0.448   0.3236]
  法向  v3 = [ 0.1002 -0.4533  0.8857]
  λ = [0.0117, 0.0029, 0.0004] | ratio_r(λ2/λ1)=0.2458, sphericity(λ3/λ1)=0.0305, linearity=0.7542, planarity=0.2153
簇 4: n=68
  主方向 v1 = [-0.4528  0.8912  0.0284]

,label,n_points,cx,cy,cz,lambda1,lambda2,lambda3,ratio_r,sphericity,...,v1x,v1y,v1z,v2x,v2y,v2z,v3x,v3y,v3z,note
0,0,1090,0.1683,-0.6016,0.1940,0.0193,0.0147,0.0001,0.7633,0.0069,...,-0.7190,-0.6667,-0.1961,0.6638,-0.5753,-0.4779,0.2058,-0.4738,0.8562,
1,1,326,0.2562,-0.2044,0.2175,0.0055,0.0022,0.0001,0.3956,0.0118,...,-0.8675,-0.0494,0.4950,0.0238,0.9898,0.1403,-0.4969,0.1335,-0.8575,
2,2,220,-0.0464,-0.2749,0.2976,0.0042,0.0018,0.0000,0.4337,0.0054,...,-0.9856,-0.1378,-0.0983,-0.1624,0.9335,0.3197,-0.0477,-0.3310,0.9424,
3,3,562,-0.1290,-0.1178,0.2788,0.0117,0.0029,0.0004,0.2458,0.0305,...,-0.5435,-0.7706,-0.3329,-0.8334,0.4480,0.3236,0.1002,-0.4533,0.8857,
4,4,68,0.0303,-0.0966,0.2958,0.0012,0.0005,0.0001,0.3867,0.0768,...,-0.4528,0.8912,0.0284,0.8893,0.4490,0.0868,-0.0646,-0.0645,0.9958,
5,5,267,-0.1999,0.1880,-0.1333,0.0104,0.0009,0.0004,0.0892,0.0405,...,-0.0582,0.4760,-0.8775,-0.1724,-0.8706,-0.4608,0.9833,-0.1244,-0.1327,
6,6,242,0.4339,0.0654,0.0424,0.0100,0.0009,0.0001,0.0932,0.0083,...,-0.9626,0.0871,0.2567,-0.1108,-0.9906,-0.0796,0.2474,-0.1051,0.9632,
7,7,136,0.1116,0.0732,0.3062,0.0023,0.0008,0.0002,0.3328,0.0835,...,-0.9151,-0.3749,-0.1485,-0.4032,0.8507,0.3373,0.0002,-0.3685,0.9296,
8,8,183,-0.1639,0.1004,0.3151,0.0043,0.0009,0.0001,0.2168,0.0223,...,-0.8269,-0.4878,-0.2799,-0.5443,0.8193,0.1801,-0.1414,-0.3013,0.9430,
9,9,178,0.0411,0.2116,0.3286,0.0039,0.0011,0.0002,0.2719,0.0389,...,-0.8412,-0.4526,0.2959,0.3757,-0.8828,-0.2820,-0.3888,0.1261,-0.9126,


In [7]:
# === Jupyter 单元格：逐簇 PCA 主成分方向与谱特征 ===

import numpy as np
import pandas as pd

# --------- 配置区（按需修改）---------
txt_path_39 = "/home/zero/mnt/sda6/split_data_instance/ptomato/quk++4/032211-39_qspp.txt"  # 你的 .txt 文件路径
IGNORE_LABEL = None    # 若想忽略某类（如茎秆=1），则设置为 1；不忽略则设 None
MIN_POINTS = 50        # 少于该点数的簇不做PCA（或只做占位）
DECIMALS = 4           # 打印小数位
SAVE_CSV = False       # 若需要保存结果为CSV，改为 True
CSV_PATH_19 = "/home/zero/mnt/sda6/split_data_instance/ptomato/quk++4/cluster_pca_summary19.csv"
# ------------------------------------
def pca_cluster(points: np.ndarray):
    """
    对 (N,3) 点云做 PCA，返回（λ按降序）特征值/向量与形态指标。
    """
    if points.shape[0] < 3:
        return dict(lambda1=np.nan, lambda2=np.nan, lambda3=np.nan,
                    v1=np.array([np.nan, np.nan, np.nan]),
                    v2=np.array([np.nan, np.nan, np.nan]),
                    v3=np.array([np.nan, np.nan, np.nan]),
                    ratio_r=np.nan, sphericity=np.nan,
                    linearity=np.nan, planarity=np.nan)
    X = points - points.mean(axis=0, keepdims=True)
    C = np.cov(X.T)                       # 3x3 协方差
    eigvals, eigvecs = np.linalg.eigh(C)  # 特征分解，eigvals 升序
    order = np.argsort(eigvals)[::-1]     # 按降序排序
    eigvals = eigvals[order]
    eigvecs = eigvecs[:, order]
    l1, l2, l3 = [float(max(v, 0.0)) for v in eigvals]
    # 防除零
    l1_eps = l1 if l1 > 1e-12 else 1e-12
    ratio_r    = l2 / l1_eps
    sphericity = l3 / l1_eps
    linearity  = (l1 - l2) / l1_eps
    planarity  = (l2 - l3) / l1_eps
    # 主方向向量（列向量）
    v1 = eigvecs[:, 0]
    v2 = eigvecs[:, 1]
    v3 = eigvecs[:, 2]
    return dict(lambda1=l1, lambda2=l2, lambda3=l3,
                v1=v1, v2=v2, v3=v3,
                ratio_r=ratio_r, sphericity=sphericity,
                linearity=linearity, planarity=planarity)

# 读取数据
arr = np.loadtxt(txt_path_39)
if arr.shape[1] < 4:
    raise ValueError("文件至少应包含 4 列：x y z label")

xyz = arr[:, :3]
labels = arr[:, 3].astype(int)
# 逐簇统计
rows = []
for lab in np.unique(labels):
    if IGNORE_LABEL is not None and lab == IGNORE_LABEL:
        continue
    pts = xyz[labels == lab]
    n = pts.shape[0]

    if n < MIN_POINTS:
        rows.append(dict(
            label=int(lab), n_points=int(n),
            cx=float(np.mean(pts[:,0])) if n>0 else np.nan,
            cy=float(np.mean(pts[:,1])) if n>0 else np.nan,
            cz=float(np.mean(pts[:,2])) if n>0 else np.nan,
            lambda1=np.nan, lambda2=np.nan, lambda3=np.nan,
            ratio_r=np.nan, sphericity=np.nan,
            linearity=np.nan, planarity=np.nan,
            v1x=np.nan, v1y=np.nan, v1z=np.nan,
            v2x=np.nan, v2y=np.nan, v2z=np.nan,
            v3x=np.nan, v3y=np.nan, v3z=np.nan,
            note="too_small"
        ))
        continue
    res = pca_cluster(pts)
    v1, v2, v3 = res['v1'], res['v2'], res['v3']
    rows.append(dict(
        label=int(lab), n_points=int(n),
        cx=float(np.mean(pts[:,0])), cy=float(np.mean(pts[:,1])), cz=float(np.mean(pts[:,2])),
        lambda1=res['lambda1'], lambda2=res['lambda2'], lambda3=res['lambda3'],
        ratio_r=res['ratio_r'], sphericity=res['sphericity'],
        linearity=res['linearity'], planarity=res['planarity'],
        v1x=v1[0], v1y=v1[1], v1z=v1[2],
        v2x=v2[0], v2y=v2[1], v2z=v2[2],
        v3x=v3[0], v3y=v3[1], v3z=v3[2],
        note=""
    ))
df = pd.DataFrame(rows).sort_values("label").reset_index(drop=True)

# 逐簇打印（主方向与指标）
np.set_printoptions(precision=DECIMALS, suppress=True)
for _, r in df.iterrows():
    lab = int(r['label']); n = int(r['n_points'])
    if r['note'] == "too_small":
        print(f"簇 {lab}: n={n} (点数过少，跳过PCA)")
        continue
    v1 = np.array([r['v1x'], r['v1y'], r['v1z']])
    v2 = np.array([r['v2x'], r['v2y'], r['v2z']])
    v3 = np.array([r['v3x'], r['v3y'], r['v3z']])
    print(f"簇 {lab}: n={n}")
    print(f"  主方向 v1 = {np.round(v1, DECIMALS)}")
    print(f"  次方向 v2 = {np.round(v2, DECIMALS)}")
    print(f"  法向  v3 = {np.round(v3, DECIMALS)}")
    print(f"  λ = [{r['lambda1']:.{DECIMALS}f}, {r['lambda2']:.{DECIMALS}f}, {r['lambda3']:.{DECIMALS}f}] "
          f"| ratio_r(λ2/λ1)={r['ratio_r']:.{DECIMALS}f}, sphericity(λ3/λ1)={r['sphericity']:.{DECIMALS}f}, "
          f"linearity={r['linearity']:.{DECIMALS}f}, planarity={r['planarity']:.{DECIMALS}f}")

# 显示汇总表（Jupyter 会渲染）
df_rounded = df.copy()
for col in df_rounded.columns:
    if df_rounded[col].dtype.kind == 'f':
        df_rounded[col] = df_rounded[col].round(DECIMALS)
df_rounded

# 如需保存CSV，取消下一行注释
# if SAVE_CSV:
#     df_rounded.to_csv(CSV_PATH, index=False, encoding="utf-8-sig")
#     print("已保存：", CSV_PATH)

簇 0: n=384
  主方向 v1 = [-0.9307  0.2451  0.2716]
  次方向 v2 = [0.3187 0.9077 0.2729]
  法向  v3 = [ 0.1796 -0.3405  0.9229]
  λ = [0.0136, 0.0044, 0.0004] | ratio_r(λ2/λ1)=0.3248, sphericity(λ3/λ1)=0.0292, linearity=0.6752, planarity=0.2956
簇 1: n=219
  主方向 v1 = [-0.0969  0.9493  0.299 ]
  次方向 v2 = [0.9951 0.0987 0.0088]
  法向  v3 = [ 0.0211 -0.2984  0.9542]
  λ = [0.0116, 0.0014, 0.0001] | ratio_r(λ2/λ1)=0.1241, sphericity(λ3/λ1)=0.0126, linearity=0.8759, planarity=0.1115
簇 2: n=222
  主方向 v1 = [-0.3349 -0.7498 -0.5707]
  次方向 v2 = [-0.8995  0.4348 -0.0434]
  法向  v3 = [-0.2807 -0.4988  0.82  ]
  λ = [0.0097, 0.0013, 0.0003] | ratio_r(λ2/λ1)=0.1334, sphericity(λ3/λ1)=0.0309, linearity=0.8666, planarity=0.1025
簇 3: n=194
  主方向 v1 = [-0.4935 -0.8698  0.0043]
  次方向 v2 = [-0.8308  0.4728  0.2937]
  法向  v3 = [ 0.2575 -0.1414  0.9559]
  λ = [0.0068, 0.0020, 0.0001] | ratio_r(λ2/λ1)=0.2928, sphericity(λ3/λ1)=0.0180, linearity=0.7072, planarity=0.2748
簇 4: n=126
  主方向 v1 = [-0.3557 -0.8674 -0.3479]
  

,label,n_points,cx,cy,cz,lambda1,lambda2,lambda3,ratio_r,sphericity,...,v1x,v1y,v1z,v2x,v2y,v2z,v3x,v3y,v3z,note
0,0,384,-0.0542,-0.0760,0.5267,0.0136,0.0044,0.0004,0.3248,0.0292,...,-0.9307,0.2451,0.2716,0.3187,0.9077,0.2729,0.1796,-0.3405,0.9229,
1,1,219,0.0331,-0.4583,0.3081,0.0116,0.0014,0.0001,0.1241,0.0126,...,-0.0969,0.9493,0.2990,0.9951,0.0987,0.0088,0.0211,-0.2984,0.9542,
2,2,222,0.1486,0.2219,0.5329,0.0097,0.0013,0.0003,0.1334,0.0309,...,-0.3349,-0.7498,-0.5707,-0.8995,0.4348,-0.0434,-0.2807,-0.4988,0.8200,
3,3,194,0.3289,-0.0248,0.4981,0.0068,0.0020,0.0001,0.2928,0.0180,...,-0.4935,-0.8698,0.0043,-0.8308,0.4728,0.2937,0.2575,-0.1414,0.9559,
4,4,126,0.1851,-0.2685,0.4414,0.0042,0.0014,0.0001,0.3281,0.0187,...,-0.3557,-0.8674,-0.3479,-0.9324,0.3037,0.1960,0.0644,-0.3941,0.9168,
5,5,626,0.5202,-0.3246,0.4266,0.0150,0.0133,0.0007,0.8837,0.0454,...,0.6620,-0.4754,-0.5795,0.6788,0.7081,0.1945,-0.3179,0.5221,-0.7914,


In [16]:
# === Jupyter 单元格：逐簇 PCA 主成分方向与谱特征 ===

import numpy as np
import pandas as pd

# --------- 配置区（按需修改）---------
txt_path_41 = "/home/zero/mnt/sda6/split_data_instance/Soybean/pred/leaf/区域+多层次谱/renumber/032212-39init_002-1.txt"  # 你的 .txt 文件路径
IGNORE_LABEL = None    # 若想忽略某类（如茎秆=1），则设置为 1；不忽略则设 None
MIN_POINTS = 50        # 少于该点数的簇不做PCA（或只做占位）
DECIMALS = 4           # 打印小数位
SAVE_CSV = False       # 若需要保存结果为CSV，改为 True
CSV_PATH_19 = "/home/zero/mnt/sda6/split_data_instance/ptomato/quk++4/cluster_pca_summary19.csv"
# ------------------------------------
def pca_cluster(points: np.ndarray):
    """
    对 (N,3) 点云做 PCA，返回（λ按降序）特征值/向量与形态指标。
    """
    if points.shape[0] < 3:
        return dict(lambda1=np.nan, lambda2=np.nan, lambda3=np.nan,
                    v1=np.array([np.nan, np.nan, np.nan]),
                    v2=np.array([np.nan, np.nan, np.nan]),
                    v3=np.array([np.nan, np.nan, np.nan]),
                    ratio_r=np.nan, sphericity=np.nan,
                    linearity=np.nan, planarity=np.nan)
    X = points - points.mean(axis=0, keepdims=True)
    C = np.cov(X.T)                       # 3x3 协方差
    eigvals, eigvecs = np.linalg.eigh(C)  # 特征分解，eigvals 升序
    order = np.argsort(eigvals)[::-1]     # 按降序排序
    eigvals = eigvals[order]
    eigvecs = eigvecs[:, order]
    l1, l2, l3 = [float(max(v, 0.0)) for v in eigvals]
    # 防除零
    l1_eps = l1 if l1 > 1e-12 else 1e-12
    ratio_r    = l2 / l1_eps
    sphericity = l3 / l1_eps
    linearity  = (l1 - l2) / l1_eps
    planarity  = (l2 - l3) / l1_eps
    # 主方向向量（列向量）
    v1 = eigvecs[:, 0]
    v2 = eigvecs[:, 1]
    v3 = eigvecs[:, 2]
    return dict(lambda1=l1, lambda2=l2, lambda3=l3,
                v1=v1, v2=v2, v3=v3,
                ratio_r=ratio_r, sphericity=sphericity,
                linearity=linearity, planarity=planarity)

# 读取数据
arr = np.loadtxt(txt_path_41)
if arr.shape[1] < 4:
    raise ValueError("文件至少应包含 4 列：x y z label")

xyz = arr[:, :3]
labels = arr[:, 3].astype(int)
# 逐簇统计
rows = []
for lab in np.unique(labels):
    if IGNORE_LABEL is not None and lab == IGNORE_LABEL:
        continue
    pts = xyz[labels == lab]
    n = pts.shape[0]

    if n < MIN_POINTS:
        rows.append(dict(
            label=int(lab), n_points=int(n),
            cx=float(np.mean(pts[:,0])) if n>0 else np.nan,
            cy=float(np.mean(pts[:,1])) if n>0 else np.nan,
            cz=float(np.mean(pts[:,2])) if n>0 else np.nan,
            lambda1=np.nan, lambda2=np.nan, lambda3=np.nan,
            ratio_r=np.nan, sphericity=np.nan,
            linearity=np.nan, planarity=np.nan,
            v1x=np.nan, v1y=np.nan, v1z=np.nan,
            v2x=np.nan, v2y=np.nan, v2z=np.nan,
            v3x=np.nan, v3y=np.nan, v3z=np.nan,
            note="too_small"
        ))
        continue
    res = pca_cluster(pts)
    v1, v2, v3 = res['v1'], res['v2'], res['v3']
    rows.append(dict(
        label=int(lab), n_points=int(n),
        cx=float(np.mean(pts[:,0])), cy=float(np.mean(pts[:,1])), cz=float(np.mean(pts[:,2])),
        lambda1=res['lambda1'], lambda2=res['lambda2'], lambda3=res['lambda3'],
        ratio_r=res['ratio_r'], sphericity=res['sphericity'],
        linearity=res['linearity'], planarity=res['planarity'],
        v1x=v1[0], v1y=v1[1], v1z=v1[2],
        v2x=v2[0], v2y=v2[1], v2z=v2[2],
        v3x=v3[0], v3y=v3[1], v3z=v3[2],
        note=""
    ))
df = pd.DataFrame(rows).sort_values("label").reset_index(drop=True)

# 逐簇打印（主方向与指标）
np.set_printoptions(precision=DECIMALS, suppress=True)
for _, r in df.iterrows():
    lab = int(r['label']); n = int(r['n_points'])
    if r['note'] == "too_small":
        print(f"簇 {lab}: n={n} (点数过少，跳过PCA)")
        continue
    v1 = np.array([r['v1x'], r['v1y'], r['v1z']])
    v2 = np.array([r['v2x'], r['v2y'], r['v2z']])
    v3 = np.array([r['v3x'], r['v3y'], r['v3z']])
    print(f"簇 {lab}: n={n}")
    print(f"  主方向 v1 = {np.round(v1, DECIMALS)}")
    print(f"  次方向 v2 = {np.round(v2, DECIMALS)}")
    print(f"  法向  v3 = {np.round(v3, DECIMALS)}")
    print(f"  λ = [{r['lambda1']:.{DECIMALS}f}, {r['lambda2']:.{DECIMALS}f}, {r['lambda3']:.{DECIMALS}f}] "
          f"| ratio_r(λ2/λ1)={r['ratio_r']:.{DECIMALS}f}, sphericity(λ3/λ1)={r['sphericity']:.{DECIMALS}f}, "
          f"linearity={r['linearity']:.{DECIMALS}f}, planarity={r['planarity']:.{DECIMALS}f}")

# 显示汇总表（Jupyter 会渲染）
df_rounded = df.copy()
for col in df_rounded.columns:
    if df_rounded[col].dtype.kind == 'f':
        df_rounded[col] = df_rounded[col].round(DECIMALS)
df_rounded

# 如需保存CSV，取消下一行注释
# if SAVE_CSV:
#     df_rounded.to_csv(CSV_PATH, index=False, encoding="utf-8-sig")
#     print("已保存：", CSV_PATH)

簇 2: n=3549
  主方向 v1 = [0.4114 0.6794 0.6076]
  次方向 v2 = [-0.7134 -0.1748  0.6786]
  法向  v3 = [ 0.5672 -0.7127  0.4128]
  λ = [0.0631, 0.0402, 0.0012] | ratio_r(λ2/λ1)=0.6375, sphericity(λ3/λ1)=0.0192, linearity=0.3625, planarity=0.6183
簇 3: n=3659
  主方向 v1 = [ 0.8538 -0.5103 -0.1029]
  次方向 v2 = [0.2647 0.2553 0.9299]
  法向  v3 = [-0.4483 -0.8212  0.353 ]
  λ = [0.0634, 0.0376, 0.0040] | ratio_r(λ2/λ1)=0.5941, sphericity(λ3/λ1)=0.0635, linearity=0.4059, planarity=0.5306
簇 4: n=1244
  主方向 v1 = [-0.516  -0.1989 -0.8332]
  次方向 v2 = [-0.8547  0.0554  0.5161]
  法向  v3 = [ 0.0565 -0.9785  0.1985]
  λ = [0.0412, 0.0126, 0.0015] | ratio_r(λ2/λ1)=0.3062, sphericity(λ3/λ1)=0.0369, linearity=0.6938, planarity=0.2694
簇 5: n=546
  主方向 v1 = [-0.9488 -0.0866 -0.3037]
  次方向 v2 = [-0.2857 -0.1745  0.9423]
  法向  v3 = [ 0.1346 -0.9808 -0.1408]
  λ = [0.0188, 0.0015, 0.0012] | ratio_r(λ2/λ1)=0.0794, sphericity(λ3/λ1)=0.0634, linearity=0.9206, planarity=0.0160
簇 6: n=50
  主方向 v1 = [-0.461  -0.4576 -0.7603]


,label,n_points,cx,cy,cz,lambda1,lambda2,lambda3,ratio_r,sphericity,...,v1x,v1y,v1z,v2x,v2y,v2z,v3x,v3y,v3z,note
0,2,3549,-0.4776,-0.1012,-0.2179,0.0631,0.0402,0.0012,0.6375,0.0192,...,0.4114,0.6794,0.6076,-0.7134,-0.1748,0.6786,0.5672,-0.7127,0.4128,
1,3,3659,0.4656,-0.1188,0.0063,0.0634,0.0376,0.0040,0.5941,0.0635,...,0.8538,-0.5103,-0.1029,0.2647,0.2553,0.9299,-0.4483,-0.8212,0.3530,
2,4,1244,-0.1356,0.4567,0.4626,0.0412,0.0126,0.0015,0.3062,0.0369,...,-0.5160,-0.1989,-0.8332,-0.8547,0.0554,0.5161,0.0565,-0.9785,0.1985,
3,5,546,0.2909,0.4139,0.3154,0.0188,0.0015,0.0012,0.0794,0.0634,...,-0.9488,-0.0866,-0.3037,-0.2857,-0.1745,0.9423,0.1346,-0.9808,-0.1408,
4,6,50,0.0282,-0.0048,0.0505,0.0025,0.0002,0.0001,0.0928,0.0574,...,-0.4610,-0.4576,-0.7603,0.6463,0.4139,-0.6410,-0.6080,0.7869,-0.1049,


In [14]:
# === Jupyter 单元格：逐簇 PCA 主成分方向与谱特征 ===

import numpy as np
import pandas as pd

# ---------
#
# 配置区（按需修改）---------
txt_path_3 = "/home/zero/mnt/sda6/split_data_instance/ptomato/cluster/renumber/032211-11init_004.txt"  # 你的 .txt 文件路径
IGNORE_LABEL = None    # 若想忽略某类（如茎秆=1），则设置为 1；不忽略则设 None
MIN_POINTS = 50        # 少于该点数的簇不做PCA（或只做占位）
DECIMALS = 4           # 打印小数位
SAVE_CSV = False       # 若需要保存结果为CSV，改为 True
CSV_PATH_19 = "/home/zero/mnt/sda6/split_data_instance/ptomato/quk++4/cluster_pca_summary3.csv"
# ------------------------------------
def pca_cluster(points: np.ndarray):
    """
    对 (N,3) 点云做 PCA，返回（λ按降序）特征值/向量与形态指标。
    """
    if points.shape[0] < 3:
        return dict(lambda1=np.nan, lambda2=np.nan, lambda3=np.nan,
                    v1=np.array([np.nan, np.nan, np.nan]),
                    v2=np.array([np.nan, np.nan, np.nan]),
                    v3=np.array([np.nan, np.nan, np.nan]),
                    ratio_r=np.nan, sphericity=np.nan,
                    linearity=np.nan, planarity=np.nan)
    X = points - points.mean(axis=0, keepdims=True)
    C = np.cov(X.T)                       # 3x3 协方差
    eigvals, eigvecs = np.linalg.eigh(C)  # 特征分解，eigvals 升序
    order = np.argsort(eigvals)[::-1]     # 按降序排序
    eigvals = eigvals[order]
    eigvecs = eigvecs[:, order]
    l1, l2, l3 = [float(max(v, 0.0)) for v in eigvals]
    # 防除零
    l1_eps = l1 if l1 > 1e-12 else 1e-12
    ratio_r    = l2 / l1_eps
    sphericity = l3 / l1_eps
    linearity  = (l1 - l2) / l1_eps
    planarity  = (l2 - l3) / l1_eps
    # 主方向向量（列向量）
    v1 = eigvecs[:, 0]
    v2 = eigvecs[:, 1]
    v3 = eigvecs[:, 2]
    return dict(lambda1=l1, lambda2=l2, lambda3=l3,
                v1=v1, v2=v2, v3=v3,
                ratio_r=ratio_r, sphericity=sphericity,
                linearity=linearity, planarity=planarity)

# 读取数据
arr = np.loadtxt(txt_path_3)
if arr.shape[1] < 4:
    raise ValueError("文件至少应包含 4 列：x y z label")

xyz = arr[:, :3]
labels = arr[:, 3].astype(int)
# 逐簇统计
rows = []
for lab in np.unique(labels):
    if IGNORE_LABEL is not None and lab == IGNORE_LABEL:
        continue
    pts = xyz[labels == lab]
    n = pts.shape[0]

    if n < MIN_POINTS:
        rows.append(dict(
            label=int(lab), n_points=int(n),
            cx=float(np.mean(pts[:,0])) if n>0 else np.nan,
            cy=float(np.mean(pts[:,1])) if n>0 else np.nan,
            cz=float(np.mean(pts[:,2])) if n>0 else np.nan,
            lambda1=np.nan, lambda2=np.nan, lambda3=np.nan,
            ratio_r=np.nan, sphericity=np.nan,
            linearity=np.nan, planarity=np.nan,
            v1x=np.nan, v1y=np.nan, v1z=np.nan,
            v2x=np.nan, v2y=np.nan, v2z=np.nan,
            v3x=np.nan, v3y=np.nan, v3z=np.nan,
            note="too_small"
        ))
        continue
    res = pca_cluster(pts)
    v1, v2, v3 = res['v1'], res['v2'], res['v3']
    rows.append(dict(
        label=int(lab), n_points=int(n),
        cx=float(np.mean(pts[:,0])), cy=float(np.mean(pts[:,1])), cz=float(np.mean(pts[:,2])),
        lambda1=res['lambda1'], lambda2=res['lambda2'], lambda3=res['lambda3'],
        ratio_r=res['ratio_r'], sphericity=res['sphericity'],
        linearity=res['linearity'], planarity=res['planarity'],
        v1x=v1[0], v1y=v1[1], v1z=v1[2],
        v2x=v2[0], v2y=v2[1], v2z=v2[2],
        v3x=v3[0], v3y=v3[1], v3z=v3[2],
        note=""
    ))
df = pd.DataFrame(rows).sort_values("label").reset_index(drop=True)

# 逐簇打印（主方向与指标）
np.set_printoptions(precision=DECIMALS, suppress=True)
for _, r in df.iterrows():
    lab = int(r['label']); n = int(r['n_points'])
    if r['note'] == "too_small":
        print(f"簇 {lab}: n={n} (点数过少，跳过PCA)")
        continue
    v1 = np.array([r['v1x'], r['v1y'], r['v1z']])
    v2 = np.array([r['v2x'], r['v2y'], r['v2z']])
    v3 = np.array([r['v3x'], r['v3y'], r['v3z']])
    print(f"簇 {lab}: n={n}")
    print(f"  主方向 v1 = {np.round(v1, DECIMALS)}")
    print(f"  次方向 v2 = {np.round(v2, DECIMALS)}")
    print(f"  法向  v3 = {np.round(v3, DECIMALS)}")
    print(f"  λ = [{r['lambda1']:.{DECIMALS}f}, {r['lambda2']:.{DECIMALS}f}, {r['lambda3']:.{DECIMALS}f}] "
          f"| ratio_r(λ2/λ1)={r['ratio_r']:.{DECIMALS}f}, sphericity(λ3/λ1)={r['sphericity']:.{DECIMALS}f}, "
          f"linearity={r['linearity']:.{DECIMALS}f}, planarity={r['planarity']:.{DECIMALS}f}")

# 显示汇总表（Jupyter 会渲染）
df_rounded = df.copy()
for col in df_rounded.columns:
    if df_rounded[col].dtype.kind == 'f':
        df_rounded[col] = df_rounded[col].round(DECIMALS)
df_rounded

# 如需保存CSV，取消下一行注释
# if SAVE_CSV:
#     df_rounded.to_csv(CSV_PATH, index=False, encoding="utf-8-sig")
#     print("已保存：", CSV_PATH)

簇 2: n=64
  主方向 v1 = [-0.3929  0.7351  0.5525]
  次方向 v2 = [ 0.7538  0.6016 -0.2644]
  法向  v3 = [ 0.5268 -0.3126  0.7904]
  λ = [0.0017, 0.0004, 0.0001] | ratio_r(λ2/λ1)=0.2145, sphericity(λ3/λ1)=0.0597, linearity=0.7855, planarity=0.1548
簇 3: n=168
  主方向 v1 = [-0.453   0.8708  0.191 ]
  次方向 v2 = [ 0.873   0.4767 -0.1029]
  法向  v3 = [ 0.1807 -0.1201  0.9762]
  λ = [0.0030, 0.0019, 0.0010] | ratio_r(λ2/λ1)=0.6510, sphericity(λ3/λ1)=0.3280, linearity=0.3490, planarity=0.3230
簇 4: n=194
  主方向 v1 = [-0.5331 -0.6935 -0.4846]
  次方向 v2 = [-0.8437  0.3936  0.3649]
  法向  v3 = [ 0.0623 -0.6035  0.795 ]
  λ = [0.0108, 0.0019, 0.0001] | ratio_r(λ2/λ1)=0.1735, sphericity(λ3/λ1)=0.0103, linearity=0.8265, planarity=0.1632
簇 5: n=155
  主方向 v1 = [-0.6334  0.7443  0.2117]
  次方向 v2 = [0.7655 0.6426 0.031 ]
  法向  v3 = [ 0.113  -0.1817  0.9768]
  λ = [0.0033, 0.0013, 0.0001] | ratio_r(λ2/λ1)=0.3979, sphericity(λ3/λ1)=0.0221, linearity=0.6021, planarity=0.3758
簇 6: n=176
  主方向 v1 = [ 0.1926 -0.5846 -0.7881]


,label,n_points,cx,cy,cz,lambda1,lambda2,lambda3,ratio_r,sphericity,...,v1x,v1y,v1z,v2x,v2y,v2z,v3x,v3y,v3z,note
0,2,64,0.0594,0.1102,0.1260,0.0017,0.0004,0.0001,0.2145,0.0597,...,-0.3929,0.7351,0.5525,0.7538,0.6016,-0.2644,0.5268,-0.3126,0.7904,
1,3,168,-0.0254,0.0343,0.1019,0.0030,0.0019,0.0010,0.6510,0.3280,...,-0.4530,0.8708,0.1910,0.8730,0.4767,-0.1029,0.1807,-0.1201,0.9762,
2,4,194,0.1587,0.0133,0.0810,0.0108,0.0019,0.0001,0.1735,0.0103,...,-0.5331,-0.6935,-0.4846,-0.8437,0.3936,0.3649,0.0623,-0.6035,0.7950,
3,5,155,0.1353,0.2160,0.1891,0.0033,0.0013,0.0001,0.3979,0.0221,...,-0.6334,0.7443,0.2117,0.7655,0.6426,0.0310,0.1130,-0.1817,0.9768,
4,6,176,0.3129,0.0888,0.0757,0.0042,0.0010,0.0001,0.2469,0.0129,...,0.1926,-0.5846,-0.7881,0.8873,0.4467,-0.1145,0.4189,-0.6773,0.6048,
5,7,181,0.2929,0.2899,0.0991,0.0045,0.0012,0.0001,0.2631,0.0161,...,0.0257,-0.7894,0.6134,0.9172,-0.2255,-0.3286,0.3977,0.5710,0.7182,
6,8,291,0.3993,0.2000,-0.0410,0.0065,0.0021,0.0002,0.3215,0.0260,...,-0.2345,0.0650,0.9699,0.0971,0.9943,-0.0432,0.9673,-0.0841,0.2395,
7,9,62,-0.0878,-0.0873,0.0272,0.0017,0.0005,0.0000,0.2682,0.0225,...,-0.9273,0.3688,0.0636,0.3659,0.8578,0.3609,-0.0785,-0.3580,0.9304,
8,10,305,-0.1899,-0.0131,0.1310,0.0069,0.0024,0.0002,0.3529,0.0255,...,-0.8216,-0.4278,-0.3768,-0.5206,0.8323,0.1901,-0.2323,-0.3524,0.9066,
9,11,87,-0.1074,0.2671,0.1322,0.0016,0.0008,0.0001,0.4732,0.0331,...,0.5473,0.7624,-0.3454,-0.5641,0.6409,0.5206,-0.6182,0.0901,-0.7808,


In [ ]:
# === Jupyter 单元格：PCA特征比值 + 1D-GMM/BIC + 2D稳定性 => 是否细分（任一满足即细分） ===
import numpy as np
import pandas as pd
from typing import Dict, Tuple, Optional, List
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture

# -------------------------
# 配置区（按需修改）
# -------------------------
txt_path = "your_initial_segmentation.txt"  # 输入文件（x y z label）
IGNORE_LABEL = 1        # 若有茎秆标签=1且不希望参与细分判定，设为1；否则设为 None
MIN_POINTS = 50         # 小于该点数的簇跳过判定（PCA/GMM均不稳定）
DECIMALS = 4            # 打印小数位

# PCA 特征比值阈值
s_th = 0.08             # sphericity = λ3/λ1 < s_th 认为足够薄
r_th = 0.75             # ratio_r = λ2/λ1 > r_th 认为主方向分散

# 1D-GMM/BIC 阈值
bic_delta = 10.0        # BIC(1)-BIC(2) > bic_delta 视为强证据支持 2 模态
sep_c = 1.2             # 均值分离度阈值（|μ1-μ2| / pooled_sigma > sep_c）
min_child_frac = 0.10   # 两侧子簇最小占比（防止把极少量尖端误判为一片叶）

# 2D-DBSCAN 稳定性阈值
min_samples_frac = 0.01     # min_samples = N * 该比例（至少2）
knn_k = 6                   # eps 基准 = 第 k 近邻距离中位数
eps_scale_grid = (0.8,1.0,1.2)  # 多阈值缩放
min_child_frac_2d = 0.10    # 2D 子簇最小占比

SAVE_CSV = False
CSV_PATH = "refine_decision_summary.csv"
# -------------------------


# -------------------------
# 工具函数：PCA 谱指标
# -------------------------
def pca_spectrum_metrics(pts: np.ndarray) -> Dict[str, float]:
    if pts.shape[0] < 3:
        return dict(lambda1=np.nan, lambda2=np.nan, lambda3=np.nan,
                    v1=np.array([np.nan,np.nan,np.nan]),
                    v2=np.array([np.nan,np.nan,np.nan]),
                    v3=np.array([np.nan,np.nan,np.nan]),
                    ratio_r=np.nan, sphericity=np.nan,
                    linearity=np.nan, planarity=np.nan)
    X = pts - pts.mean(axis=0, keepdims=True)
    C = np.cov(X.T)
    eigvals, eigvecs = np.linalg.eigh(C)     # 升序
    order = np.argsort(eigvals)[::-1]        # 降序
    eigvals = eigvals[order]
    eigvecs = eigvecs[:, order]
    l1, l2, l3 = [float(max(v, 0.0)) for v in eigvals]
    l1_eps = l1 if l1 > 1e-12 else 1e-12

    ratio_r    = l2 / l1_eps
    sphericity = l3 / l1_eps
    linearity  = (l1 - l2) / l1_eps
    planarity  = (l2 - l3) / l1_eps

    return dict(lambda1=l1, lambda2=l2, lambda3=l3,
                v1=eigvecs[:,0], v2=eigvecs[:,1], v3=eigvecs[:,2],
                ratio_r=ratio_r, sphericity=sphericity,
                linearity=linearity, planarity=planarity)

# -------------------------
# 1D-GMM/BIC（沿主轴 v1 的一维双峰检验）
# -------------------------
def one_d_gmm_bic_test(pts: np.ndarray, v1: np.ndarray, min_child_frac: float = 0.10,
                       bic_delta: float = 10.0, sep_c: float = 1.2) -> Dict[str, object]:
    """
    把点投影到主轴 v1：t = <x - mean, v1>
    用 GMM(1) vs GMM(2) 比较 BIC；并检查两均值分离度与子簇占比。
    返回：
      - bic1, bic2, bic_gain
      - sep, weights, means
      - pass_gmm: 是否通过 1D-GMM 条件
    """
    N = pts.shape[0]
    if N < 20 or not np.all(np.isfinite(v1)):
        return dict(bic1=np.inf, bic2=np.inf, bic_gain=0.0,
                    sep=0.0, weights=None, means=None, pass_gmm=False)

    Xc = pts - pts.mean(axis=0, keepdims=True)
    v1n = v1 / (np.linalg.norm(v1) + 1e-12)
    t = Xc.dot(v1n).reshape(-1,1)

    try:
        g1 = GaussianMixture(n_components=1, covariance_type='full', random_state=0).fit(t)
        g2 = GaussianMixture(n_components=2, covariance_type='full', random_state=0).fit(t)
        bic1 = g1.bic(t); bic2 = g2.bic(t)
        bic_gain = bic1 - bic2  # >0 越大越支持2模态

        means = np.sort(g2.means_.flatten())
        weights = g2.weights_
        # 组内方差（两簇的加权）
        sigmas = np.sqrt(np.array([g2.covariances_[i].squeeze() for i in range(2)]))
        pooled = np.sqrt(np.sum(weights * (sigmas**2)))
        sep = abs(means[1] - means[0]) / (pooled + 1e-12)

        # 子簇占比检查（两侧都要不小于 min_child_frac）
        w_ok = np.all(weights >= min_child_frac)

        pass_gmm = (bic_gain > bic_delta) and (sep > sep_c) and w_ok
        return dict(bic1=float(bic1), bic2=float(bic2), bic_gain=float(bic_gain),
                    sep=float(sep), weights=weights, means=means, pass_gmm=bool(pass_gmm))
    except Exception:
        return dict(bic1=np.inf, bic2=np.inf, bic_gain=0.0,
                    sep=0.0, weights=None, means=None, pass_gmm=False)

# -------------------------
# 2D-DBSCAN 稳定性（PCA前两主成分）
# -------------------------
def two_d_stability_test(pts: np.ndarray,
                         min_samples_frac: float = 0.01,
                         knn_k: int = 6,
                         eps_scale_grid: Tuple[float, ...] = (0.8,1.0,1.2),
                         min_child_frac: float = 0.10) -> Dict[str, object]:
    N = pts.shape[0]
    if N < 10:
        return dict(stable_multi=False, n_clusters_grid=[], best_labels=None)

    P2 = PCA(n_components=2).fit_transform(pts)
    min_samples = max(2, int(np.ceil(N * min_samples_frac)))

    nn = NearestNeighbors(n_neighbors=min(knn_k, N)).fit(P2)
    dists, _ = nn.kneighbors(P2)
    base_eps = float(np.median(dists[:, -1]))

    n_clusters_grid = []
    best_labels = None
    multi_count = 0

    for s in eps_scale_grid:
        eps = max(1e-6, s * base_eps)
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(P2)
        labels = db.labels_
        uniq = [c for c in np.unique(labels) if c >= 0]
        good_clusters = [c for c in uniq if np.sum(labels == c) >= max(2, int(N * min_child_frac))]
        n_clusters_grid.append(len(good_clusters))
        if len(good_clusters) >= 2:
            multi_count += 1
            best_labels = labels

    stable_multi = (multi_count >= (len(eps_scale_grid) // 2 + 1))
    return dict(stable_multi=bool(stable_multi),
                n_clusters_grid=n_clusters_grid,
                best_labels=best_labels)

# -------------------------
# 主逻辑：读取数据，逐簇计算并判定
# -------------------------
arr = np.loadtxt(txt_path)
if arr.shape[1] < 4:
    raise ValueError("文件至少需要 4 列：x y z label")

xyz = arr[:, :3]
labels = arr[:, 3].astype(int)

rows = []
np.set_printoptions(precision=DECIMALS, suppress=True)

for lab in np.unique(labels):
    if IGNORE_LABEL is not None and lab == IGNORE_LABEL:
        # 可跳过茎秆等非叶片标签
        continue
    pts = xyz[labels == lab]
    n = pts.shape[0]

    if n < MIN_POINTS:
        print(f"簇 {lab}: n={n} (点数过少，跳过细分判定)")
        rows.append(dict(
            label=int(lab), n_points=int(n),
            ratio_r=np.nan, sphericity=np.nan,
            linearity=np.nan, planarity=np.nan,
            # 1D-GMM
            bic1=np.nan, bic2=np.nan, bic_gain=np.nan, sep=np.nan,
            w1=np.nan, w2=np.nan,
            # 2D
            ncl_grid=str([]), stable_multi=False,
            need_refine=False, reason="too_small"
        ))
        continue

    # --- PCA 谱 ---
    pm = pca_spectrum_metrics(pts)
    ratio_r = pm['ratio_r']; sphericity = pm['sphericity']
    linearity = pm['linearity']; planarity = pm['planarity']
    v1 = pm['v1']

    cond_feature = (np.isfinite(ratio_r) and np.isfinite(sphericity)
                    and (ratio_r > r_th) and (sphericity < s_th))

    # --- 1D-GMM/BIC ---
    gmm = one_d_gmm_bic_test(pts, v1, min_child_frac=min_child_frac,
                             bic_delta=bic_delta, sep_c=sep_c)
    cond_gmm = gmm['pass_gmm']
    w1 = (float(gmm['weights'][0]) if isinstance(gmm['weights'], np.ndarray) else np.nan)
    w2 = (float(gmm['weights'][1]) if isinstance(gmm['weights'], np.ndarray) else np.nan)

    # --- 2D 稳定性 ---
    s2d = two_d_stability_test(pts, min_samples_frac=min_samples_frac,
                               knn_k=knn_k, eps_scale_grid=eps_scale_grid,
                               min_child_frac=min_child_frac_2d)
    cond_2d = bool(s2d['stable_multi'])

    # --- 最终判定（任意一项满足即细分） ---
    need_refine = bool(cond_feature or cond_gmm or cond_2d)

    # 打印
    print(f"簇 {lab}: n={n}")
    print(f"  PCA: ratio_r(λ2/λ1)={ratio_r:.{DECIMALS}f}, sphericity(λ3/λ1)={sphericity:.{DECIMALS}f}, "
          f"linearity={linearity:.{DECIMALS}f}, planarity={planarity:.{DECIMALS}f} | 触发={cond_feature}")
    print(f"  1D-GMM: BIC1={gmm['bic1']:.2f}, BIC2={gmm['bic2']:.2f}, ΔBIC={gmm['bic_gain']:.2f}, "
          f"sep={gmm['sep']:.{DECIMALS}f}, weights={gmm['weights']} | 触发={cond_gmm}")
    print(f"  2D稳定性: n_clusters_grid={s2d['n_clusters_grid']} -> stable_multi={cond_2d}")
    print(f"  ==> 是否需要细分：{'是' if need_refine else '否'}\n")

    # 记录
    rows.append(dict(
        label=int(lab), n_points=int(n),
        ratio_r=float(ratio_r), sphericity=float(sphericity),
        linearity=float(linearity), planarity=float(planarity),
        bic1=float(gmm['bic1']), bic2=float(gmm['bic2']),
        bic_gain=float(gmm['bic_gain']), sep=float(gmm['sep']),
        w1=w1, w2=w2,
        ncl_grid=str(s2d['n_clusters_grid']), stable_multi=bool(cond_2d),
        cond_feature=bool(cond_feature), cond_gmm=bool(cond_gmm), cond_2d=bool(cond_2d),
        need_refine=bool(need_refine)
    ))

# 汇总表
df = pd.DataFrame(rows).sort_values("label").reset_index(drop=True)
df_round = df.copy()
for c in df_round.columns:
    if df_round[c].dtype.kind == 'f':
        df_round[c] = df_round[c].round(DECIMALS)

# 显示 DataFrame
df_round

# 可选：保存 CSV
# if SAVE_CSV:
#     df_round.to_csv(CSV_PATH, index=False, encoding="utf-8-sig")
#     print("已保存：", CSV_PATH)

In [3]:
import numpy as np

def compute_cluster_variance(subset):
    center = np.mean(subset, axis=0)
    distances = np.linalg.norm(subset - center, axis=1)
    variance = np.var(distances)
    return variance

def calculate_variances(points, labels):
    unique_labels = np.unique(labels)
    variance_dict = {}

    for label in unique_labels:
        subset = points[labels == label]
        variance = compute_cluster_variance(subset)
        variance_dict[label] = variance
    sorted_variances = dict(sorted(variance_dict.items(), key=lambda item: item[1]))

    return sorted_variances

In [2]:
import numpy as np
from sklearn.metrics import silhouette_samples
def calculate_silhouette_scores(points, labels):
    # 计算每个点的轮廓系数
    silhouette_vals = silhouette_samples(points, labels)
    # 计算每个簇的平均轮廓系数
    unique_labels = np.unique(labels)
    silhouette_dict = {}
    for label in unique_labels:
        cluster_silhouette_vals = silhouette_vals[labels == label]
        silhouette_dict[label] = np.mean(cluster_silhouette_vals)
    return silhouette_dict

def should_split_cluster(silhouette_score, threshold=0.1):
    return silhouette_score < threshold

In [5]:
corn_12 = np.loadtxt('/mnt/data1/split_data_instance/Corn/corn01/select/init/renumber/ini/032213-12_0.032.txt')
points_corn_12 = corn_12[:,0:3]
labels_corn_12 = corn_12[:,3] # 示例标签

v_corn_12 = calculate_variances(points_corn_12, labels_corn_12)
print("variances!")
print(v_corn_12)

s_corn_12 = calculate_silhouette_scores(points_corn_12, labels_corn_12)

for label, score in s_corn_12.items():
    if should_split_cluster(score):
        print(f"簇 {label} 可能需要进一步分割，轮廓系数: {score:.2f}")
    else:
        print(f"簇 {label} 不需要进一步分割，轮廓系数: {score:.2f}")


variances!
{3.0: 0.0026462820939672717, 9.0: 0.0037347601354576526, 10.0: 0.004272834121833077, 7.0: 0.005168529637106315, 5.0: 0.005668577902190757, 11.0: 0.005930684077572051, 8.0: 0.00742329063621777, 6.0: 0.008042465076489617, 4.0: 0.008457542759132861, 2.0: 0.009733525750689372}
簇 2.0 可能需要进一步分割，轮廓系数: 0.01
簇 3.0 不需要进一步分割，轮廓系数: 0.58
簇 4.0 不需要进一步分割，轮廓系数: 0.45
簇 5.0 不需要进一步分割，轮廓系数: 0.39
簇 6.0 不需要进一步分割，轮廓系数: 0.33
簇 7.0 不需要进一步分割，轮廓系数: 0.29
簇 8.0 不需要进一步分割，轮廓系数: 0.36
簇 9.0 不需要进一步分割，轮廓系数: 0.49
簇 10.0 不需要进一步分割，轮廓系数: 0.47
簇 11.0 不需要进一步分割，轮廓系数: 0.36


In [6]:
corn_23 = np.loadtxt('/mnt/data1/split_data_instance/Corn/corn01/select/init/renumber/ini/032213-23_0.032.txt')
points_corn_23 = corn_23[:,0:3]
labels_corn_23 = corn_23[:,3] # 示例标签

v_corn_23 = calculate_variances(points_corn_23, labels_corn_23)
print("variances!")
print(v_corn_23)

s_corn_23 = calculate_silhouette_scores(points_corn_23, labels_corn_23)

for label, score in s_corn_23.items():
    if should_split_cluster(score):
        print(f"簇 {label} 可能需要进一步分割，轮廓系数: {score:.2f}")
    else:
        print(f"簇 {label} 不需要进一步分割，轮廓系数: {score:.2f}")

variances!
{6.0: 0.0020451215550210404, 4.0: 0.004220110443592676, 7.0: 0.0048200363206615145, 5.0: 0.006146998213086683, 2.0: 0.006908994698067565, 3.0: 0.02043164556900511}
簇 2.0 不需要进一步分割，轮廓系数: 0.40
簇 3.0 不需要进一步分割，轮廓系数: 0.17
簇 4.0 不需要进一步分割，轮廓系数: 0.47
簇 5.0 不需要进一步分割，轮廓系数: 0.39
簇 6.0 不需要进一步分割，轮廓系数: 0.63
簇 7.0 不需要进一步分割，轮廓系数: 0.38


In [7]:
corn_34 = np.loadtxt('/mnt/data1/split_data_instance/Corn/corn01/select/init/renumber/ini/032213-34_0.045.txt')
points_corn_34 = corn_34[:,0:3]
labels_corn_34 = corn_34[:,3] # 示例标签

v_corn_34 = calculate_variances(points_corn_34, labels_corn_34)
print("variances!")
print(v_corn_34)

s_corn_34 = calculate_silhouette_scores(points_corn_34, labels_corn_34)

for label, score in s_corn_34.items():
    if should_split_cluster(score):
        print(f"簇 {label} 可能需要进一步分割，轮廓系数: {score:.2f}")
    else:
        print(f"簇 {label} 不需要进一步分割，轮廓系数: {score:.2f}")

variances!
{2.0: 0.0027689247366617037, 5.0: 0.006599322172904072, 4.0: 0.015008373364901174, 3.0: 0.021896105744309877}
簇 2.0 不需要进一步分割，轮廓系数: 0.74
簇 3.0 不需要进一步分割，轮廓系数: 0.21
簇 4.0 不需要进一步分割，轮廓系数: 0.38
簇 5.0 不需要进一步分割，轮廓系数: 0.58


In [18]:
corn_40 = np.loadtxt('/mnt/data1/split_data_instance/Corn/corn01/select/init/renumber/ini/032213-137_0.02.txt')
points_corn_40 = corn_40[:,0:3]
labels_corn_40 = corn_40[:,3] # 示例标签

v_corn_40 = calculate_variances(points_corn_40, labels_corn_40)
print("variances!")
print(v_corn_40)

s_corn_40 = calculate_silhouette_scores(points_corn_40, labels_corn_40)

for label, score in s_corn_40.items():
    if should_split_cluster(score):
        print(f"簇 {label} 可能需要进一步分割，轮廓系数: {score:.2f}")
    else:
        print(f"簇 {label} 不需要进一步分割，轮廓系数: {score:.2f}")

variances!
{5.0: 0.006775553347473738, 7.0: 0.00722837960808662, 4.0: 0.00821397238151359, 6.0: 0.012972125644964907, 2.0: 0.013800272649279806, 3.0: 0.016891222010707113}
簇 2.0 不需要进一步分割，轮廓系数: 0.51
簇 3.0 不需要进一步分割，轮廓系数: 0.18
簇 4.0 不需要进一步分割，轮廓系数: 0.70
簇 5.0 不需要进一步分割，轮廓系数: 0.39
簇 6.0 不需要进一步分割，轮廓系数: 0.21
簇 7.0 不需要进一步分割，轮廓系数: 0.54


In [26]:
potato_ = np.loadtxt('/mnt/data1/split_data_instance/potato/leaf/init/renumber/032215-82_0.02.txt')
points_potato_ = potato_[:,0:3]
labels_potato_ = potato_[:,3] # 示例标签

v_potato_ = calculate_variances(points_potato_, labels_potato_)
print("variances!")
print(v_potato_)

s_potato_ = calculate_silhouette_scores(points_potato_, labels_potato_)

for label, score in s_potato_.items():
    if should_split_cluster(score):
        print(f"簇 {label} 可能需要进一步分割，轮廓系数: {score:.2f}")
    else:
        print(f"簇 {label} 不需要进一步分割，轮廓系数: {score:.2f}")

variances!
{8.0: 0.0012938826944236888, 4.0: 0.001523531077722287, 7.0: 0.0016945583343860558, 3.0: 0.002098748974079803, 6.0: 0.0021741798966285345, 2.0: 0.002791291938193747, 5.0: 0.013584518305818343}
簇 2.0 不需要进一步分割，轮廓系数: 0.29
簇 3.0 不需要进一步分割，轮廓系数: 0.53
簇 4.0 不需要进一步分割，轮廓系数: 0.75
簇 5.0 不需要进一步分割，轮廓系数: 0.62
簇 6.0 不需要进一步分割，轮廓系数: 0.52
簇 7.0 不需要进一步分割，轮廓系数: 0.64
簇 8.0 不需要进一步分割，轮廓系数: 0.49
